In [1]:
from vllm import LLM, SamplingParams
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

model_args = {
            'model': 'Qwen/Qwen2-7B',
            'gpu_memory_utilization': 0.8,
            'trust_remote_code': False, 
            'revision': 'main',
            'dtype': 'float16', 
            'seed': 1234,
            'max_model_len': None, 
            'tensor_parallel_size': 1, 
            'pipeline_parallel_size': 1, 
            'swap_space': 4, 
        }

llm = LLM(**model_args)


/data/qiujiang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-13 18:38:07 __init__.py:190] Automatically detected platform cuda.


2025-02-13 18:38:07,649	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 02-13 18:38:09 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-13 18:38:14 config.py:542] This model supports multiple tasks: {'classify', 'generate', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 02-13 18:38:14 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-13 18:38:14 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-13 18:38:14 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2-7B', speculative_config=None, tokenizer='Qwen/Qwen2-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=main, override_neuron_config=None, tokenizer_revision=main, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_format=a

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:03,  1.07s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:02<00:02,  1.03s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:03<00:01,  1.06s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.07s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.06s/it]



INFO 02-13 18:38:22 model_runner.py:1115] Loading model weights took 14.2717 GB
INFO 02-13 18:38:23 worker.py:267] Memory profiling takes 0.66 seconds
INFO 02-13 18:38:23 worker.py:267] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.80) = 63.40GiB
INFO 02-13 18:38:23 worker.py:267] model weights take 14.27GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 47.64GiB.
INFO 02-13 18:38:23 executor_base.py:110] # CUDA blocks: 55749, # CPU blocks: 4681
INFO 02-13 18:38:23 executor_base.py:115] Maximum concurrency for 131072 tokens per request: 6.81x
INFO 02-13 18:38:25 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:24<00:00,  1.44it/s]

INFO 02-13 18:38:50 model_runner.py:1562] Graph capturing finished in 24 secs, took 0.19 GiB
INFO 02-13 18:38:50 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 27.81 seconds


In [ ]:
from vllm.transformers_utils.tokenizer import get_tokenizer

vllm_tokenizer = get_tokenizer(
               "Qwen/Qwen2-7B",
               tokenizer_mode="auto",
               trust_remote_code=False,
               tokenizer_revision="main",
            )
# 这个text让sglang第二次生成就基本上对齐
# input_text = "Question: Fred was preparing for a party to be held in four days.  So, he made 24 gallons of root beer on the first day and put them in the refrigerator cooler.  But later that evening, his children discovered the delicious nectar and robbed the cooler, drinking 4 of those gallons of root beer.  On the second day, his wife Barbie also discovered the root beer and accidentally spilled 7 gallons. On the third day, Fred's friend Ronnie visited Fred's house and helped himself to the root beer, further reducing the amount remaining by 5 gallons.  On the fourth day, 3 people showed up for the party.  If Fred and the others shared the remaining root beer equally, how much was available for each to drink during the party?\nAnswer:"
# 这个text，sglang和vllm生成一次就对齐了
# input_text = "Question: A farmer is buying feed for his horses. He buys a variety of hay, oats, carrots and sugar cubes. Since sugar cubes are a rare treat, he only buys two 1-pound boxes of them for the whole stable. He only wants enough carrots to feed the horses while the vegetables are fresh, so he buys four 12-pound bags. Hay is the main diet of his horses, so he buys forty-two 75-pound bales. Oats are a staple to supplement the hay, so he buys twenty 65-pound sacks. If his farm truck can carry 2250 pounds at a time, how many trips does the farmer need to transport all the feed?\nAnswer:"
# 这个text对齐次数一直在变，但是也有齐的时候
input_text = "Question: The great dragon, Perg, sat high atop mount Farbo, breathing fire upon anything within a distance of 1000 feet.  Polly could throw the gold javelin, the only known weapon that could sleigh the dragon, for a distance of 400 feet, well within the reach of the dragon's flames.  But when Polly held the sapphire gemstone, she could throw the javelin three times farther than when not holding the gemstone. If holding the gemstone, how far outside of the reach of the dragon's flames could Polly stand and still hit the dragon with the gold javelin?\nAnswer:"

input_tokens = vllm_tokenizer.encode(input_text)

vllm_sampling_params = SamplingParams(
    n=1, 
    presence_penalty=0.0, 
    frequency_penalty=0.0, 
    repetition_penalty=1.0, 
    temperature=1.0, 
    top_p=1.0, 
    top_k=-1, 
    min_p=0.0, 
    seed=None, 
    stop=['Question=', 'Question', '=', '<|endoftext|>'], 
    stop_token_ids=[], 
    bad_words=[], 
    include_stop_str_in_output=False, 
    ignore_eos=False, 
    max_tokens=256, 
    min_tokens=0, 
    logprobs=0, 
    prompt_logprobs=None, 
    skip_special_tokens=True, 
    spaces_between_special_tokens=True, 
    truncate_prompt_tokens=None, 
    guided_decoding=None
    )

vllm_output = llm.generate(prompt_token_ids=input_tokens,
                sampling_params=vllm_sampling_params)
vllm_output = vllm_output[0]
output_token_ids = [outputs.token_ids for outputs in vllm_output.outputs]
logprobs = [output.logprobs for output in vllm_output.outputs] or []
logprobs = [logprob[token_id].logprob for token_id, logprob in zip(output_token_ids[0], logprobs[0])]
result = [output.text for output in vllm_output.outputs]
input_token_ids = vllm_output.prompt_token_ids
print(f"Tokenized Input: {input_tokens}")
print(result)
print(logprobs)
print(vllm_output)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s, est. speed input: 193.94 toks/s, output: 79.34 toks/s]

Tokenized Input: [14582, 25, 576, 2244, 25105, 11, 393, 2375, 11, 7578, 1550, 45988, 6470, 13452, 749, 11, 25938, 3940, 5193, 4113, 2878, 264, 6010, 315, 220, 16, 15, 15, 15, 7541, 13, 220, 93417, 1410, 2510, 279, 6623, 1258, 32730, 11, 279, 1172, 3881, 10288, 429, 1410, 19747, 1090, 279, 25105, 11, 369, 264, 6010, 315, 220, 19, 15, 15, 7541, 11, 1632, 2878, 279, 5545, 315, 279, 25105, 594, 38785, 13, 220, 1988, 979, 93417, 5644, 279, 274, 55105, 18747, 10812, 11, 1340, 1410, 2510, 279, 1258, 32730, 2326, 3039, 42626, 1091, 979, 537, 9963, 279, 18747, 10812, 13, 1416, 9963, 279, 18747, 10812, 11, 1246, 3041, 4889, 315, 279, 5545, 315, 279, 25105, 594, 38785, 1410, 93417, 2498, 323, 2058, 4201, 279, 25105, 448, 279, 6623, 1258, 32730, 5267, 16141, 25]
[' When Polly is not holding the gemstone, she can throw the javelin 400 feet. \nHowever, when she holds the gemstone, she can throw the javelin three times farther, so that would be 400 feet * 3 ']
[-1.498100757598877, -0.1620449423789978

: 

In [ ]:
# from vllm.distributed.parallel_state import destroy_distributed_environment, destroy_model_parallel
# import torch
# import gc

# destroy_model_parallel()
# del llm.llm_engine.model_executor.driver_worker
# gc.collect()
# torch.cuda.empty_cache()
exit(0)

: 